In [0]:
import yaml
from pyspark.sql import SparkSession
import os
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# 1. Konfiguracja Środowiska
try:
    ENV = dbutils.widgets.get("env_name")
except Exception:
    ENV = 'TEST'
    
# 2. Wczytanie Konfiguracji
try:
    with open('../../config/config.yaml', 'r') as file:
        full_config = yaml.safe_load(file)
except FileNotFoundError:
    print("BŁĄD: Plik 'config.yaml' nie został znaleziony! Sprawdź ścieżkę.")
    raise

CFG = full_config.get(ENV)
if not CFG:
    raise ValueError(f"Nie znaleziono konfiguracji dla środowiska: {ENV} w pliku YAML.")

catalog_name = CFG['catalog_name']
schema_name = CFG['schema_name']
volume_name = CFG['volume_name']

base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_bronze_data"
print(f"Ścieżka do tabeli Delta Bronze: {base_output_directory}")

In [0]:
#wczytywanie danych z tabeli delta
try:
    # Używamy formatu "delta" do odczytu folderu Delta Lake
    df_bronze = (
        spark.read.format("delta") 
        .load(base_output_directory) # Ładowanie ze ścieżki do folderu Delta
        .withColumn("Date", col("Date").cast("date"))
    )
    
    print(f"Pomyślnie wczytano dane. Liczba wierszy: {df_bronze.count()}")
   
except Exception as e:
    print(f"BŁĄD: Wystąpił problem podczas ładowania danych: {e}")

In [0]:


# 2. Zastosowanie optymalizacji typów danych
df_optimized = (
    df_bronze
    .withColumn("Date", col("Date").cast("date")) # Data: pozostaje 'date'
    
    # Zmiana z 'double' na 'float' (lub 'FloatType()')
    .withColumn("Open", col("Open").cast(FloatType()))
    .withColumn("High", col("High").cast(FloatType()))
    .withColumn("Low", col("Low").cast(FloatType()))
    .withColumn("Close", col("Close").cast(FloatType()))
    
    # Zmiana z 'long' na 'integer' (lub 'IntegerType()')
    .withColumn("Volume", col("Volume").cast(IntegerType())) 
    # 'Ticket' i 'company_name' pozostają 'string'
)

# Podmiana oryginalnej zmiennej na zoptymalizowany DataFrame
df_bronze = df_optimized

print(f"Zoptymalizowano schemat.")
df_bronze.printSchema()

df_bronze.show(100)


In [0]:
# Minimalna Walidacja Krytycznych Kolumn Pod Kątem nulli
critical_null_count = df_bronze.filter(
    F.col("Date").isNull() | 
    F.col("Ticket").isNull() | 
    F.col("Close").isNull()
).count()

if critical_null_count > 0:
    # Wyrzuć błąd, który zatrzyma potok ETL i wymaga interwencji
    raise ValueError(f"BŁĄD QA: Znaleziono {critical_null_count} wierszy z NULLami w kolumnach krytycznych (Date, Ticket, Close). Potok zatrzymany.")

In [0]:
#sprawdzanie duplikatów po Date&Ticket
deduplication_key = ["Date", "Ticket"]
total_rows = df_bronze.count()
df_unique = df_bronze.dropDuplicates(subset=deduplication_key)
unique_rows = df_unique.count()
duplicate_count = total_rows - unique_rows
duplicate_percentage = (duplicate_count / total_rows) * 100

print(f"--- Raport Duplikatów ---")
print(f"Całkowita liczba wierszy: {total_rows}")
print(f"Liczba unikalnych wierszy: {unique_rows}")
print(f"Liczba znalezionych duplikatów: {duplicate_count}")
print(f"Procent duplikatów: {duplicate_percentage}%")

# IMPLEMENTACJA ZATRZYMANIA KODU
if total_rows != unique_rows:
    # Używamy ValueError, aby zatrzymać potok i poinformować o błędzie
    raise ValueError(
        f"BŁĄD QA: Znaleziono duplikaty w kluczu (Date, Ticket)! "
        f"Łącznie: {duplicate_count} wierszy ({duplicate_percentage}%). "
        f"Potok ETL został zatrzymany w celu inspekcji danych źródłowych."
    )
    
# Jeśli duplikatów nie ma, kontynuujemy i nadpisujemy (lub po prostu używamy) df_unique
df_bronze = df_unique 
print("WALIDACJA DUPLIKATÓW: ZAKOŃCZONA POWODZENIEM. Przechodzę dalej.")


In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType # Potrzebne do rzutowania

# Zakładam, że df_bronze, F, Window, FloatType są zaimportowane i dostępne

PRECISION = 4

# Definicja Okna: Działamy w ramach każdego Tickera, sortując po dacie
window_spec = (
    Window.partitionBy("Ticket")
    .orderBy("Date")
)

df_silver = (
    df_bronze
    .withColumn(
        "Previous_Close", 
        # 1. Obliczenie ceny zamknięcia z poprzedniego dnia handlowego
        F.lag(F.col("Close"), 1).over(window_spec)
    )
    .withColumn(
        "Daily_Return_Pct", 
        # 2. Obliczenie procentowej zmiany i jawne zaokrąglenie do PRECISION miejsc
        F.round(
            ((F.col("Close") - F.col("Previous_Close")) / F.col("Previous_Close")) * 100, 
            PRECISION # Użycie stałej 4
        ).cast(FloatType())
    )
    .drop("Previous_Close") # 3. Usunięcie tymczasowej kolumny
)

# Podmiana oryginalnej zmiennej na DataFrame z warstwy Silver
df_silver

print(f"Pomyślnie obliczono i zaokrąglono 'Daily_Return_Pct' do {PRECISION} miejsc po przecinku.")
df_silver.show(20)

In [0]:
df_silver = (
    df_silver
    .withColumn(
        "Year", 
        F.year(F.col("Date")) # Wyodrębnia rok (np. 2023)
    )
    .withColumn(
        "Quarter", 
        F.quarter(F.col("Date")) # Wyodrębnia kwartał (1, 2, 3 lub 4)
    )
    .withColumn(
        "Month", 
        F.month(F.col("Date")) # Wyodrębnia miesiąc (1 do 12)
    )
    .withColumn(
        "WeekOfYear", 
        # Wyodrębnia numer tygodnia w roku (1 do 53)
        F.weekofyear(F.col("Date")) 
    )
)

print("Pomyślnie dodano kolumny Year, Quarter, Month, WeekOfYear.")

# Wyświetlenie nowych kolumn
df_silver.show(100)

In [0]:
# --- Okresy SMA ---
SMA_PERIODS = [20, 50, 200]
base_window_spec = (
    Window.partitionBy("Ticket")
    .orderBy("Date")
)

# Inicjalizujemy DataFrame, na którym będziemy pracować (obecnie df_bronze zawiera już Daily_Return_Pct)
df_silver_sma = df_silver

# --- Iteracyjne tworzenie kolumn SMA ---
for N in SMA_PERIODS:
    # 1. Definicja Specyfikacji Okna dla danego okresu N
    # rowsBetween(-(N-1), 0) oznacza: od N-1 wierszy wstecz do bieżącego wiersza (0)
    window_spec_n = base_window_spec.rowsBetween(-(N - 1), 0)

    column_name = f"SMA_{N}"
    
    # 2. Obliczenie Średniej Kroczącej (SMA)
    df_silver_sma = df_silver_sma.withColumn(
        column_name,
        # Używamy funkcji avg() na kolumnie 'Close' w ramach zdefiniowanego okna
        F.avg(F.col("Close")).over(window_spec_n).cast(FloatType())
    )
df_silver = df_silver_sma

print(f"Pomyślnie dodano kolumny SMA dla okresów: {SMA_PERIODS}")

df_silver.show(100)
